# Python Inotebook for model fitting

### Initial steps of importing libraries and reading data

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc  
from sklearn import cross_validation
from sklearn import svm
from sklearn.cross_validation import cross_val_score 

In [2]:

number = preprocessing.LabelEncoder() 



In [5]:
#loading the Data
train=pd.read_csv('C:\Users\Krish\Google Drive\Coursera\coursera\Projects\uptake\uptake\processed_data\imputed_train.csv') 
#train.head() 

### Convert categorical variable to numpy arrays 

In [12]:

def convert(data):
    number = preprocessing.LabelEncoder()
    data['profession'] = number.fit_transform(data.profession)
    data['marital'] = number.fit_transform(data.marital) 
    data['default'] = number.fit_transform(data.default)
    data['schooling'] = number.fit_transform(data.schooling)
    data['housing'] = number.fit_transform(data.housing)
    data['loan'] = number.fit_transform(data.loan)
    data['contact'] = number.fit_transform(data.contact)
    data['month'] = number.fit_transform(data.month)
    data['day_of_week'] = number.fit_transform(data.day_of_week)
    data['poutcome'] = number.fit_transform(data.poutcome) 
    #data['responded'] =number.fit_transform(data.responded)
    return data




       

In [7]:
train=convert(train)

In [10]:
train[0:10]
married 1
divorced 0 
single 2

,custAge,profession,marital,schooling,default,housing,loan,contact,month,day_of_week,...,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,pmonths,pastEmail,responded
0,34.000000,0,2,6,0,0,1,0,0,4,...,0,1,-1.8,93.075,-47.1,1.498,5099.1,999,0,0
1,31.000000,7,2,3,0,0,0,0,3,2,...,0,1,1.4,93.918,-42.7,4.968,5228.1,999,0,0
2,34.531295,0,2,3,0,0,0,1,4,3,...,0,1,1.4,94.465,-41.8,4.961,5228.1,999,0,0
3,52.000000,0,0,6,0,1,0,0,3,3,...,0,1,1.4,93.918,-42.7,4.962,5228.1,999,0,0
4,39.000000,1,2,2,0,1,0,0,3,3,...,0,1,1.4,93.918,-42.7,4.961,5228.1,999,0,0
5,40.000000,2,1,6,0,1,0,1,4,2,...,0,1,1.4,94.465,-41.8,4.866,5228.1,999,0,0
6,50.000000,9,2,5,0,0,0,0,3,3,...,0,1,1.4,93.918,-42.7,4.961,5228.1,999,0,0
7,41.000000,9,1,5,0,0,0,0,8,2,...,0,1,-3.4,92.431,-26.9,0.741,5017.5,999,0,0
8,23.000000,1,2,0,0,1,0,1,4,0,...,0,1,1.4,94.465,-41.8,4.959,5228.1,999,0,0
9,29.000000,9,1,5,0,1,0,0,1,1,...,0,1,1.4,93.444,-36.1,4.965,5228.1,999,0,0


### Splitting Predictors (X_train) and Target Variable (Y_train)

In [131]:
Y_train=train['responded']
X_train=train.drop(['responded'],axis=1) 

 ### Spliting  the data set into training (x_train,y_train) and validation set (x_validate,y_validate)

In [132]:
train['is_train'] = np.random.uniform(0, 1, len(train)) <= .75
train, validate = train[train['is_train']==True], train[train['is_train']==False] 

In [133]:
y_train = train['responded']
x_train=train.drop(['responded','is_train'],axis=1) 
x_validate=validate.drop(['responded','is_train'],axis=1) 
y_validate=validate['responded'] 

 ## Model 1 logistic regression model

In [134]:
lg = LogisticRegression()
lg.fit(x_train, y_train) 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [135]:
 Disbursed_lg=lg.predict_proba(x_validate) 

### Evaluating model from K fold validation & AUC Metrics 

In [136]:
# Area under the curve
fpr, tpr, _ = roc_curve(y_validate, Disbursed_lg[:,1])
roc_auc = auc(fpr, tpr)
print roc_auc  

# evaluate the model using 10-fold cross-validation
scores = cross_val_score(LogisticRegression(), X_train, Y_train, scoring='accuracy', cv=10)
print scores
print scores.mean()  

0.80004021313
[ 0.90390879  0.8990228   0.91042345  0.91530945  0.90553746  0.90553746
  0.90390879  0.91205212  0.90065147  0.89250814]
0.904885993485


## Model 2 Random Forest Model

In [137]:
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(x_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## Evaluating model from K fold validation & AUC Metrics

In [138]:
disbursed = rf.predict_proba(x_validate)
fpr, tpr, _ = roc_curve(y_validate, disbursed[:,1])
roc_auc = auc(fpr, tpr) 
print roc_auc   

scores = cross_val_score(RandomForestClassifier(),X_train, Y_train, scoring='accuracy', cv=10)
print scores
print scores.mean() 

0.808304849033
[ 0.89739414  0.89576547  0.89413681  0.89576547  0.90228013  0.89576547
  0.90065147  0.90065147  0.88925081  0.88599349]
0.895765472313


## Model 3: Support Vector Machines

In [156]:
clf = svm.SVC(kernel='linear', C=1).fit(x_train, y_train) 

## Evaluating model from K fold validation & AUC Metrics

In [140]:
print(clf.score(x_validate, y_validate))  

clf = svm.SVC(kernel='linear', C=1)
scores = cross_validation.cross_val_score(
clf, X_train,Y_train, cv=5)
print(scores)
print(scores.mean()) 

0.895860284605
[ 0.89739414  0.90960912  0.89983713  0.90309446  0.89576547]
0.901140065147


# Predicting Response Variable in TestingCandidate.csv  

In [11]:
#loading the Data
test=pd.read_csv('C:\Users\Krish\Google Drive\Coursera\coursera\Projects\uptake\uptake\processed_data/impute_testingCandidate.csv')
#train.head()

In [15]:
test=convert(test) 
test[0:10]

,custAge,profession,marital,schooling,default,housing,loan,contact,month,day_of_week,...,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,pmonths,pastEmail
0,36.650000,0,1,6,0,0,1,0,9,4,...,999,1,0,-1.1,94.199,-37.5,0.886,4963.6,999.0,2
1,35.000000,7,1,3,0,0,0,0,9,3,...,3,1,2,-3.4,92.379,-29.8,0.788,5017.5,0.1,2
2,50.000000,1,1,5,0,1,0,0,6,2,...,999,1,0,-1.8,92.893,-46.2,1.327,5099.1,999.0,2
3,30.000000,0,2,6,0,0,0,0,1,4,...,999,0,1,1.4,93.444,-36.1,4.964,5228.1,999.0,0
4,39.000000,7,0,3,0,1,0,0,7,3,...,999,0,1,-0.1,93.200,-42.0,4.153,5195.8,999.0,0
5,30.079118,8,2,3,0,1,0,0,3,2,...,999,0,1,1.4,93.918,-42.7,4.958,5228.1,999.0,0
6,36.000000,3,1,0,0,1,0,0,0,4,...,999,0,1,-1.8,93.075,-47.1,1.415,5099.1,999.0,0
7,30.000000,0,2,2,0,1,0,1,3,2,...,999,0,1,1.4,93.918,-42.7,4.963,5228.1,999.0,0
8,35.075244,1,2,1,0,1,0,1,6,4,...,999,0,1,1.1,93.994,-36.4,4.859,5191.0,999.0,0
9,43.361765,6,1,2,0,0,0,0,1,0,...,999,0,1,1.4,93.444,-36.1,4.966,5228.1,999.0,0


# Predictions from Model 1 (Logistic Regression)

In [148]:
 predict_logistic=lg.predict(test) 

In [153]:
predict_logistic[:20]

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Predictions from Model 2 (Random Forest)

In [152]:
predict_randomForest = rf.predict(test)


In [154]:
predict_randomForest[:20]

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

# Predictions from Model 3 (Support Vector Machine) 

In [157]:
predict_SVM = clf.predict(test)

In [160]:
predict_SVM[:20]

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

# Merging All the columns and taking the votes of all three Models as final accuracy

In [161]:
print(type(test))
print(type(predict_SVM))

<class 'pandas.core.frame.DataFrame'>
<type 'numpy.ndarray'>


In [168]:
test['predict_SVM']=predict_SVM
test['predict_randomForest']=predict_randomForest
test['predict_logistic']= predict_logistic

In [169]:
test.head()

,custAge,profession,marital,schooling,default,housing,loan,contact,month,day_of_week,...,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,pmonths,pastEmail,predict_SVM,predict_randomForest,predict_logistic
0,36.65,0,1,6,0,0,1,0,9,4,...,-1.1,94.199,-37.5,0.886,4963.6,999.0,2,0,0,0
1,35.00,7,1,3,0,0,0,0,9,3,...,-3.4,92.379,-29.8,0.788,5017.5,0.1,2,1,1,1
2,50.00,1,1,5,0,1,0,0,6,2,...,-1.8,92.893,-46.2,1.327,5099.1,999.0,2,0,0,0
3,30.00,0,2,6,0,0,0,0,1,4,...,1.4,93.444,-36.1,4.964,5228.1,999.0,0,0,0,0
4,39.00,7,0,3,0,1,0,0,7,3,...,-0.1,93.200,-42.0,4.153,5195.8,999.0,0,0,0,0


In [170]:
test.to_csv("Final_response_testing_data.csv", sep=',', encoding='utf-8') 